In [1]:
import os
import sys
import pandas as pd
import numpy as np
from scipy.io import loadmat
from sklearn.model_selection import train_test_split

In [4]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

In [6]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [9]:
from time import time
random_state=np.random.RandomState(23)

In [10]:
mat_file_list=['arrhythmia.mat','cardio.mat','glass.mat','ionosphere.mat','letter.mat','lympho.mat','mnist.mat','musk.mat',
              'optdigits.mat','pendigits.mat','pima.mat','satellite.mat','satimage-2.mat','shuttle.mat','vertebral.mat','vowels.mat',
              'wbc.mat']

In [25]:
for i in mat_file_list:
    print('Processing',i,'..')
    mat=loadmat(os.path.join('data',i))
    X=mat['X']
    y=mat['y'].ravel()
    outlier_frac=np.count_nonzero(y)/len(y)
    outlier_per=round(outlier_frac*100, ndigits=4)
    
    roc_list=[i[:-4],X.shape[0],X.shape[1],outlier_per]
    prn_list=[i[:-4],X.shape[0],X.shape[1],outlier_per]
    time_list=[i[:-4],X.shape[0],X.shape[1],outlier_per]
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.4,random_state=0)
    X_train_norm,X_test_norm=standardizer(X_train,X_test)
    
    classifiers={'Angle based outlier detection(ABOD)': ABOD(contamination=outlier_frac),
                'Cluster based local outlier factor(CBLOF)': CBLOF(contamination=outlier_frac,random_state=0),
                'Feature bagging': FeatureBagging(contamination=outlier_frac,random_state=0),
                'Histogram based outlier detection(HBOS)': HBOS(contamination=outlier_frac),
                'Isolation Forest': IForest(contamination=outlier_frac,random_state=0),
                'KNN': KNN(contamination=outlier_frac),
                'Local outlier factor': LOF(contamination=outlier_frac),
                'Minimum covariance determinant(MCD)': MCD(contamination=outlier_frac,random_state=0),
                'One class SVM(OCSVM)': OCSVM(contamination=outlier_frac),
                'Principal component analysis(PCA)': PCA(contamination=outlier_frac,random_state=0)}
    
    for clf_name, clf in classifiers.items():
        t0=time()
        clf.fit(X_train_norm)
        test_score=clf.decision_function(X_test_norm)
        t1=time()
        duration=round(t1-t0, ndigits=4)
        time_list.append(duration)
        
        roc=round(roc_auc_score(y_test,test_score), ndigits=4)
        prn=round(precision_n_scores(y_test,test_score), ndigits=4)
        
        print('{clf_name} ROC:{roc}, precision @ rank n:{prn}, execution time: {duration}s'.format(clf_name=clf_name, roc=roc,
                                                                                                  prn=prn, duration=duration))
        roc_list.append(roc)
        prn_list.append(prn)
        
    temp_df=pd.DataFrame(time_list).transpose()
#     temp_df.columns=df_columns
    time_df=pd.DataFrame()
    time_df=pd.concat([time_df,temp_df],axis=0)
    
    temp_df=pd.DataFrame(roc_list).transpose()
#     temp_df.columns=df_columns
    roc_df=pd.DataFrame()
    roc_df=pd.concat([roc_df,temp_df],axis=0)
    
    temp_df=pd.DataFrame(prn_list).transpose()
#     temp_df.columns=df_columns
    prn_df=pd.DataFrame()
    prn_df=pd.concat([prn_df,temp_df],axis=0)
        
        

Processing arrhythmia.mat ..
Angle based outlier detection(ABOD) ROC:0.8035, precision @ rank n:0.3462, execution time: 0.2622s
Cluster based local outlier factor(CBLOF) ROC:0.8102, precision @ rank n:0.5385, execution time: 0.3068s
Feature bagging ROC:0.804, precision @ rank n:0.5, execution time: 1.0216s
Histogram based outlier detection(HBOS) ROC:0.8608, precision @ rank n:0.5385, execution time: 0.1203s
Isolation Forest ROC:0.8092, precision @ rank n:0.5385, execution time: 0.5676s
KNN ROC:0.8176, precision @ rank n:0.5, execution time: 0.2117s
Local outlier factor ROC:0.8082, precision @ rank n:0.5, execution time: 0.1899s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:622: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum covariance determinant(MCD) ROC:0.8464, precision @ rank n:0.5, execution time: 1.4259s
One class SVM(OCSVM) ROC:0.8102, precision @ rank n:0.5, execution time: 0.1179s
Principal component analysis(PCA) ROC:0.8164, precision @ rank n:0.5, execution time: 0.1337s
Processing cardio.mat ..
Angle based outlier detection(ABOD) ROC:0.5783, precision @ rank n:0.2388, execution time: 1.0709s
Cluster based local outlier factor(CBLOF) ROC:0.7488, precision @ rank n:0.2836, execution time: 0.3162s
Feature bagging ROC:0.6352, precision @ rank n:0.1791, execution time: 1.6513s
Histogram based outlier detection(HBOS) ROC:0.8435, precision @ rank n:0.4627, execution time: 0.016s
Isolation Forest ROC:0.9292, precision @ rank n:0.4925, execution time: 0.6086s
KNN ROC:0.7309, precision @ rank n:0.2836, execution time: 0.3831s
Local outlier factor ROC:0.6078, precision @ rank n:0.194, execution time: 0.208s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:622: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (-152.153389035376364 > -251.133615685855716)
  % (det, previous_det), RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (-153.124633442918082 > -248.857697717310401)
  % (det, previous_det), RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (-151.033690762432428 > -248.529549161825088)
  % (det, previous_det), RuntimeWarning)


Minimum covariance determinant(MCD) ROC:0.7765, precision @ rank n:0.403, execution time: 1.3542s
One class SVM(OCSVM) ROC:0.928, precision @ rank n:0.4179, execution time: 0.1786s
Principal component analysis(PCA) ROC:0.9454, precision @ rank n:0.5522, execution time: 0.009s
Processing glass.mat ..
Angle based outlier detection(ABOD) ROC:0.6546, precision @ rank n:0.0, execution time: 0.0945s
Cluster based local outlier factor(CBLOF) ROC:0.8594, precision @ rank n:0.0, execution time: 0.0741s
Feature bagging ROC:0.8996, precision @ rank n:0.3333, execution time: 0.0918s
Histogram based outlier detection(HBOS) ROC:0.6707, precision @ rank n:0.0, execution time: 0.007s
Isolation Forest ROC:0.6466, precision @ rank n:0.0, execution time: 0.4069s
KNN ROC:0.8072, precision @ rank n:0.0, execution time: 0.014s
Local outlier factor ROC:0.9116, precision @ rank n:0.3333, execution time: 0.004s
Minimum covariance determinant(MCD) ROC:0.759, precision @ rank n:0.0, execution time: 0.1098s
One c

...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (-49.017135173908585 > -75.821118329156164)
  % (det, previous_det), RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (-48.964475974812970 > -76.042798063569563)
  % (det, previous_det), RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (-49.091020181851121 > -75.551173091310517)
  % (det, previous_det), RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (-49.863005166647042 > -74.599558709585608)
  % (det, previous_det), RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (-49.907890752606832 > -76.473093442706869)
  % (det, previous_det), RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (-50.616020052782680 > -77.029215127899889)
  % (det, previous_det), RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (-50.779036047925729 > -77.235385663255698)
  % (det, previous_det), RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (-50.296580960568448 > -76.300006300128842)
  % (det, previous_det), RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det 

Minimum covariance determinant(MCD) ROC:0.9908, precision @ rank n:0.7615, execution time: 23.5402s
One class SVM(OCSVM) ROC:0.9911, precision @ rank n:0.9519, execution time: 97.5249s
Principal component analysis(PCA) ROC:0.989, precision @ rank n:0.9498, execution time: 0.0958s
Processing vertebral.mat ..
Angle based outlier detection(ABOD) ROC:0.4272, precision @ rank n:0.0, execution time: 0.1476s
Cluster based local outlier factor(CBLOF) ROC:0.442, precision @ rank n:0.0, execution time: 0.1167s
Feature bagging ROC:0.5012, precision @ rank n:0.0, execution time: 0.0955s
Histogram based outlier detection(HBOS) ROC:0.3778, precision @ rank n:0.0, execution time: 0.0052s
Isolation Forest ROC:0.4337, precision @ rank n:0.0, execution time: 0.4037s
KNN ROC:0.437, precision @ rank n:0.0, execution time: 0.017s
Local outlier factor ROC:0.4807, precision @ rank n:0.0, execution time: 0.0061s
Minimum covariance determinant(MCD) ROC:0.4461, precision @ rank n:0.0, execution time: 0.1455s
On

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (-23.585498360269710 > -41.658348437252911)
  % (det, previous_det), RuntimeWarning)



Principal component analysis(PCA) ROC:0.4272, precision @ rank n:0.0, execution time: 0.006s
Processing vowels.mat ..
Angle based outlier detection(ABOD) ROC:0.9538, precision @ rank n:0.5556, execution time: 0.6092s
Cluster based local outlier factor(CBLOF) ROC:0.882, precision @ rank n:0.2778, execution time: 0.1836s
Feature bagging ROC:0.9213, precision @ rank n:0.2778, execution time: 0.6196s
Histogram based outlier detection(HBOS) ROC:0.6272, precision @ rank n:0.1667, execution time: 0.007s
Isolation Forest ROC:0.7735, precision @ rank n:0.3333, execution time: 0.5599s
KNN ROC:0.9751, precision @ rank n:0.5556, execution time: 0.1719s
Local outlier factor ROC:0.9443, precision @ rank n:0.3333, execution time: 0.072s
Minimum covariance determinant(MCD) ROC:0.6951, precision @ rank n:0.0, execution time: 1.8715s
One class SVM(OCSVM) ROC:0.7615, precision @ rank n:0.3333, execution time: 0.0968s
Principal component analysis(PCA) ROC:0.5799, precision @ rank n:0.1111, execution time